### Imports

In [3]:
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, Callback
from datetime import datetime
import keras.backend as K
import extras.ourUtils
import numpy as np
import Models


### Init

In [ ]:
batch_size = 40
full_train = True
path = '/home/jaskmo/Documents/programering/02456DomainAdaptation/'
source_data = path + 'taperImages/pysNetData'
target_data = path + 'taperImages/hData'

## Get data as generators

#### Train data

In [ ]:

train_gen_source = datagen.flow_from_directory(source_data + '/train', target_size=(224, 224), 
                                               batch_size=batch_size, class_mode='categorical', shuffle=True)

train_gen_target = datagen.flow_from_directory(target_data + '/train', target_size=(224, 224), 
                                               batch_size=batch_size, class_mode='categorical', shuffle=True)

train_stepE = np.floor_divide(train_gen_source.n, batch_size)

#### validation data

In [ ]:
valid_gen_source = datagen.flow_from_directory(source_data + '/validation', target_size=(224, 224), 
                                               batch_size=batch_size), class_mode='categorical', shuffle=True)
valid_gen_target = datagen.flow_from_directory(target_data + '/validation', target_size=(224, 224), 
                                               batch_size=batch_size, class_mode='categorical', shuffle=True)

val_stepE = np.floor_divide(valid_gen_source.n, batch_size)

#### test data

In [ ]:
test_gen_source = datagen.flow_from_directory(source_data + '/test', target_size=(224, 224), 
                                               batch_size=batch_size, class_mode='categorical', shuffle=True)
test_gen_target = datagen.flow_from_directory(target_data + '/test', target_size=(224, 224), 
                                               batch_size=batch_size, class_mode='categorical', shuffle=True)

test_stepE = np.floor_divide(test_gen_source.n, batch_size)

### get model

In [ ]:
current_model = Models.DA_model(batch_size=batch_size)

### Callbacks

In [4]:
now = datetime.now()
csv_logger = CSVLogger('/media/jaskmo/ELEK/bme/Project02456/trainingLog/DA_Model' + 
                           str(now.day) + '-' + str(now.month) + '-' + str(now.year) + '_' + 
                           str(now.hour) + str(now.minute) + '.log')

class FlipControle(Callback):
    def __init__(self, alphaIn):
        self.alpha = alphaIn
        print(K.get_value(lamFunk))
        
    def on_epoch_end(self, epoch, logs={}):
        p = (epoch+1)/nrEpochs
        K.set_value(self.alpha, (2/(1+np.exp(-10*p)))-1)
        print(K.get_value(lamFunk))

### Fit the S!@¤